##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [1]:

import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.13434112, -0.384212  , -0.26214778,  0.5265097 , -0.14735883,
        -0.19019142,  0.14275897, -0.11332513, -0.15776035,  0.28087613]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.08802661, 0.06856406, 0.07746549, 0.1704582 , 0.08688814,
        0.08324508, 0.11613338, 0.08989616, 0.08598907, 0.13333389]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.4859662

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - accuracy: 0.9152 - loss: 0.2919
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - accuracy: 0.9584 - loss: 0.1395
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - accuracy: 0.9683 - loss: 0.1054
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - accuracy: 0.9732 - loss: 0.0858
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - accuracy: 0.9773 - loss: 0.0738


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)".

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - accuracy: 0.9781 - loss: 0.0698


[0.06977338343858719, 0.9781000018119812]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.90181902e-06, 1.25813381e-06, 3.52549599e-04, 2.12747278e-03,
        2.28633801e-09, 8.35946480e-07, 2.32711558e-11, 9.97498930e-01,
        6.99479642e-06, 1.00168227e-05],
       [8.76202833e-10, 8.68096424e-04, 9.99099612e-01, 2.72651905e-05,
        5.50540516e-14, 1.33334436e-06, 2.20669733e-07, 2.12782171e-13,
        3.50215032e-06, 1.39996836e-12],
       [2.33562628e-06, 9.98115778e-01, 5.74756450e-05, 1.37293064e-05,
        1.38068091e-04, 3.75814161e-05, 1.19144152e-05, 1.39380002e-03,
        2.25944241e-04, 3.41510645e-06],
       [9.99862790e-01, 1.19948265e-08, 4.52526619e-05, 3.05697057e-07,
        7.73600561e-07, 4.38868892e-06, 7.97248940e-05, 1.92090738e-06,
        9.00229153e-08, 4.84500106e-06],
       [1.94251311e-06, 9.68277014e-09, 1.60139862e-05, 9.44598142e-08,
        9.99133885e-01, 7.15917622e-07, 9.23761581e-06, 1.00361256e-04,
        1.80072664e-06, 7.35916256e-04]], dtype=float32)>